In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('hp_raw.csv')
print(df.columns)
savefile_rate = 'HealthExpenditureRate.csv'

df = df.drop(columns=['Country Name', 'Indicator Name', 'Indicator Code'])
df = df.set_index(['Country Code'])
# linear interpolate missing values
df = df.T.interpolate(limit_direction='both').T.reset_index()
df = df.dropna(axis=0)
df = df.set_index('Country Code')
# df.to_csv(savefile_rate)

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019', '2020'],
      dtype='object')


In [3]:
# rate = pd.read_csv(savefile_rate).set_index(['Country Code'])
rate = df
rate = rate/100

latest_available_year = 2019
# given the volatility in estimates, we use rolling average to get the growth rate
rate2000 = rate[[str(year) for year in range(2000,2009,1)]].mean(axis=1)
rate2010 = rate[[str(year) for year in range(2010,2019,1)]].mean(axis=1)
exp = (rate2010/rate2000)**0.1

# follow the projection rule in the write-up
upperCap = 0.1244
lowerCap = 0.03

# now do the projection
for country in rate.index:
    for year in range(latest_available_year+1,2051,1):
        if (rate.loc[country,str(year-1)]<upperCap) and (rate.loc[country,str(year-1)]>lowerCap):
            rate.loc[country,str(year)] = rate.loc[country,str(year-1)]*exp.loc[country]
        else:
            rate.loc[country,str(year)] = rate.loc[country,str(year-1)]

rate = round(rate, 10) 
rate.to_csv(savefile_rate)

In [4]:

# gdp and population use the output from the GDP and population_labor section
PPP = True
if PPP:
    gdp = pd.read_csv('../GDP/GDP_ppp.csv').set_index(['Country Code'])
else:
    gdp = pd.read_csv('../GDP/GDP.csv').set_index(['Country Code'])
pop = pd.read_csv('../population_labor/UNDP/population_total.csv')


if PPP:
    savefile = 'hepc_ppp.csv'
else:
    savefile = 'hepc.csv'
# 

In [5]:
he = gdp*rate[[str(year) for year in range(2010,2051,1)]]
he = he.dropna(axis=0, how='all')

pop = pop.groupby('Country Code').sum()
pop = pop[[str(year) for year in range(2010,2051,1)]]
hepc = (he[[str(year) for year in range(2010,2051,1)]]/pop).dropna(axis=0, how='all')
hepc.to_csv(savefile)
print(len(hepc))

182
